In [1]:
from langchain_core.prompts.chat import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv

In [2]:
load_dotenv(override=True)

True

In [3]:
reflection_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
             """
             You are a viral twitter influencer grading a tweet. Generate critique and recommendations for the user's tweet.
             Always provide a detailed recommendations, including request for length, virality, style, etc.
             """
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [4]:
generation_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system", 
            """
            You are a twitter techie influencer assistant tasked with writing excellent twitter post.
            Generate the best twitter post possible for the users request.
            If the user provides critique, respond with a revised version of your previous attempts
            """
        ),
        MessagesPlaceholder(variable_name="messages")
    ]
)

In [5]:
llm = ChatOpenAI(model = "gpt-4o-mini")

In [6]:
reflection_chain = reflection_prompt | llm

In [7]:
generation_chain = generation_prompt | llm

In [8]:
from langchain_core.messages.base import BaseMessage
from langchain_core.messages.human import HumanMessage
from typing import Sequence, List
from langgraph.graph import MessageGraph, END, START

In [9]:
def generation_chain_processing(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    return generation_chain.invoke({"messages": state})

In [10]:
def reflection_chain_processing(state: Sequence[BaseMessage]) -> List[BaseMessage]:
    response = reflection_chain.invoke({"messages": state})
    return [HumanMessage(content = response.content)]

In [11]:
flow = MessageGraph()

In [12]:
flow.add_node("generation_node", generation_chain_processing)
flow.add_node("reflection_node", reflection_chain_processing)
flow.add_edge(START, "generation_node")

In [13]:
def should_reflect_or_end(state: Sequence[BaseMessage]) -> str:
    if len(state) > 6:
        return "should end"

    return "should reflect"

In [15]:
flow.add_conditional_edges("generation_node", should_reflect_or_end, 
                           {
                               "should end": END, 
                               "should reflect": "reflection_node"
                           })

In [16]:
flow.add_edge("reflection_node", "generation_node")

In [17]:
builder = flow.compile()

In [18]:
builder.get_graph().draw_mermaid_png(output_file_path ="flow.png", )

b'\x89PNG\r\n\x1a\n\x00\x00\x00\rIHDR\x00\x00\x010\x00\x00\x01\x11\x08\x02\x00\x00\x00sil7\x00\x00\x00\x01sRGB\x00\xae\xce\x1c\xe9\x00\x00 \x00IDATx\x9c\xed\xddg@\x13\xc9\xdb\x00\xf0I/$\xf4\xde\x04\xa4\x08\x08\x82 v\x14TDQ,`/\xd8\xc5\xeeYN\xcf\xb3\x97\xf3PO\xb1\x8b\xfd\x10=\xd4;\x15+*\x16T\xac\x08\x08\x02\x8aH\x93\xaa\xf4\xf4\xfe~X\xdf\xc8\x1f\x01\x01\x13v\x13\xe6\xf7)\xd9\xdd\xd9}R\x9e\x9d\x9d-38\x99L\x06 \x08\xc2\x06<\xda\x01@\x10\xf4\rLH\x08\xc2\x10\x98\x90\x10\x84!0!!\x08C`BB\x10\x86\xc0\x84\x84 \x0c!\xa2\x1d\x00\xa4D\xa5y|N\xad\x98S+\x91\x88eB\x9e\x14\xedp~\x8cB\xc3\x13H8\r-"C\x93hhIA;\x1c\x14\xe0\xe0uH5#\x93\x81\x8c\x17\xb5\xb9o\xd9y\x19\xdc\x0e\x9d\xe8D2\x8e\xaeI\xd41 \x0bx\x12\xb4C\xfb1\n\r_Y&\xe2\xd4\x8a\tx\\n\x06\xc7\xba\xb3\x86\xb5\xb3\x86\x83\x07\x13\xed\xb8\xda\x0eLH\xb5\xf2\xfa^UJ|\xb5Mg\rkg\r+g\r\xb4\xc3\xf9)\x12\x91,\'\x9d\x93\x9f\xc9\xcdIc\xf7\x1e\xae\xef\xdcS\x13\xed\x88\xda\x02LH5Q\x98\xc5\xbb\xf5w\x89sO\xad^\xc3\xf4\xd0\x8eE\xc1\x04<\xe9\xd3k\xe5e\x9f\x04~\x93\x8ct\

In [19]:
input_msg = HumanMessage(content = """
Make this tweet better: "
@langchainAI - newly Tool Calling feature is seroiusly underrated.

After a long wait, it's here - making the implementation of agents across different models with function calling - super easy.

made a video convering their newest blog post
""")

In [20]:
response = builder.invoke(input_msg)

In [23]:
print(respnse[-1].content)

🚀 Big news from @langchainAI! Their new Tool Calling feature is a game changer, effortlessly streamlining workflows for agents across models! 💡 I've already seen improvements in my own projects! Check out my video diving into their latest blog post. How will this change YOUR projects? 📹✨ #AI #LangChain #TechTrends


In [ ]:
print()